In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# clone and install Mask2Former
!git clone https://github.com/facebookresearch/Mask2Former.git
%cd Mask2Former
!pip install -U opencv-python
!pip install git+https://github.com/cocodataset/panopticapi.git
!pip install -r requirements.txt
%cd mask2former/modeling/pixel_decoder/ops
!python setup.py build install
%cd ../../../../

fatal: destination path 'Mask2Former' already exists and is not an empty directory.
/content/Mask2Former
  Cloning https://github.com/cocodataset/panopticapi.git to /tmp/pip-req-build-viu9kht6
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/panopticapi.git /tmp/pip-req-build-viu9kht6
  Resolved https://github.com/cocodataset/panopticapi.git to commit 7bb4655548f98f3fedc07bf37e9040a992b054b0
  Preparing metadata (setup.py) ... done
/content/Mask2Former/mask2former/modeling/pixel_decoder/ops
running build
running build_py
running build_ext
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:500: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:415: UserWarning: The detected CUDA version (12.2) has a minor version mi

In [3]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-q9fifwdc
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-q9fifwdc
  Resolved https://github.com/facebookresearch/detectron2.git to commit 0ae803b1449cd2d3f8fa1b7c0f59356db10b3083
  Preparing metadata (setup.py) ... done


In [4]:
%cd mask2former/modeling/pixel_decoder/ops

/content/Mask2Former/mask2former/modeling/pixel_decoder/ops


In [5]:
!sh make.sh

running build
running build_py
running build_ext
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:500: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:415: UserWarning: The detected CUDA version (12.2) has a minor version mismatch with the version that was used to compile PyTorch (12.1). Most likely this shouldn't be a problem.
  warnings.warn(CUDA_MISMATCH_WARN.format(cuda_str_version, torch.version.cuda))
/usr/local/lib/python3.10/dist-packages/torch/utils/cpp_extension.py:425: UserWarning: There are no x86_64-linux-gnu-g++ version bounds defined for CUDA version 12.2
  warnings.warn(f'There are no {compiler_name} version bounds defined for CUDA version {cuda_str_version}')
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_dist

In [6]:
%cd ../../../..

/content/Mask2Former


In [7]:
# import Mask2Former project
from mask2former import add_maskformer2_config

In [8]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [9]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()
setup_logger(name="mask2former")

# import some common libraries
import numpy as np
import cv2
import torch
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog
from detectron2.projects.deeplab import add_deeplab_config

In [10]:
data_dir_path = "/content/drive/MyDrive/instseg/data/"

In [12]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("taco_train", {}, data_dir_path + "annotations_0_train.json", data_dir_path + "images/")
register_coco_instances("taco_val", {}, data_dir_path + "annotations_0_val.json", data_dir_path + "images/")
register_coco_instances("taco_test", {}, data_dir_path + "annotations_0_test.json", data_dir_path + "images/")

In [11]:
import json

# Load your JSON file
with open('/content/drive/MyDrive/instseg/data/annotations_0_train.json') as f:
    data = json.load(f)

# Generate unique IDs
unique_id = 0
for annotation in data['annotations']:
    annotation['id'] = unique_id
    unique_id += 1

# Save the corrected JSON back to file
with open('/content/drive/MyDrive/instseg/data/annotations_0_train.json', 'w') as f:
    json.dump(data, f, indent=4)


# Fine-tune

In [14]:
from detectron2.data import DatasetMapper, build_detection_train_loader
from detectron2.data import detection_utils as utils
from detectron2.structures import PolygonMasks
import copy
import torch  # Import torch to convert images to tensors
from argparse import ArgumentParser
from detectron2.config import get_cfg
from detectron2.data import build_detection_train_loader
from detectron2.engine import DefaultTrainer
from detectron2.projects.deeplab import add_deeplab_config
import detectron2.utils.comm as comm
from detectron2.utils.logger import setup_logger
from mask2former import (
    MaskFormerInstanceDatasetMapper,
    InstanceSegEvaluator,
    add_maskformer2_config,
)

In [15]:
class Trainer(DefaultTrainer):
    """
    Extension of the Trainer class adapted to MaskFormer.
    """
    @classmethod
    def build_train_loader(cls, cfg):
        if cfg.INPUT.DATASET_MAPPER_NAME == "mask_former_instance":
            mapper = MaskFormerInstanceDatasetMapper(cfg, True)
            return build_detection_train_loader(cfg, mapper=mapper)

In [16]:
import os
from detectron2.config import get_cfg


cfg = get_cfg()
add_deeplab_config(cfg)
add_maskformer2_config(cfg)

cfg.merge_from_file("configs/coco/instance-segmentation/maskformer2_R50_bs16_50ep.yaml")

In [19]:
print(cfg)

CUDNN_BENCHMARK: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  FILTER_EMPTY_ANNOTATIONS: True
  NUM_WORKERS: 4
  REPEAT_SQRT: True
  REPEAT_THRESHOLD: 0.0
  SAMPLER_TRAIN: TrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: ()
  PROPOSAL_FILES_TRAIN: ()
  TEST: ('coco_2017_val',)
  TRAIN: ('coco_2017_train',)
GLOBAL:
  HACK: 1.0
INPUT:
  COLOR_AUG_SSD: False
  CROP:
    ENABLED: False
    SINGLE_CATEGORY_MAX_AREA: 1.0
    SIZE: [0.9, 0.9]
    TYPE: relative_range
  DATASET_MAPPER_NAME: coco_instance_lsj
  FORMAT: RGB
  IMAGE_SIZE: 1024
  MASK_FORMAT: polygon
  MAX_SCALE: 2.0
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SCALE: 0.1
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: (800,)
  MIN_SIZE_TRAIN_SAMPLING: choice
  RANDOM_FLIP: horizontal
  SIZE_DIVISIBILITY: -1
MODEL:
  ANCHOR_GENERATOR:
    ANGLES: [[-90, 0, 90]]
    ASPECT_RATIOS: [[0.5, 1.0, 2.0]]
    NAME: DefaultAnchorGenerator
    OFFSET: 0.0
    SI

In [23]:
cfg.DATASETS.TRAIN = ("taco_train",)
cfg.DATASETS.TEST = ()

cfg.DATALOADER.NUM_WORKERS = 2

cfg.INPUT.DATASET_MAPPER_NAME = "mask_former_instance"

cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 60
cfg.MODEL.WEIGHTS = "https://dl.fbaipublicfiles.com/maskformer/mask2former/coco/instance/maskformer2_R50_bs16_50ep/model_final_3c8ec9.pkl"
cfg.SOLVER.CLIP_GRADIENTS.CLIP_TYPE = "value"
cfg.SOLVER.CLIP_GRADIENTS.CLIP_VALUE = 1.0
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 300
cfg.SOLVER.STEPS = []

cfg.OUTPUT_DIR = "./output"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

trainer = Trainer(cfg)
trainer.resume_or_load(resume=False)

[05/10 13:44:05 d2.engine.defaults]: Model:
MaskFormer(
  (backbone): ResNet(
    (stem): BasicStem(
      (conv1): Conv2d(
        3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
    )
    (res2): Sequential(
      (0): BottleneckBlock(
        (shortcut): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
        )
        (conv1): Conv2d(
          64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv2): Conv2d(
          64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
        (conv3): Conv2d(
          64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
          (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
      

criterion.empty_weight
sem_seg_head.predictor.class_embed.{bias, weight}


In [24]:
trainer.train()

[05/10 13:44:14 d2.engine.train_loop]: Starting training from iteration 0
[05/10 13:44:38 d2.utils.events]:  eta: 0:03:56  iter: 19  total_loss: 102.9  loss_ce: 5.905  loss_mask: 0.1864  loss_dice: 1.948  loss_ce_0: 8.838  loss_mask_0: 0.1969  loss_dice_0: 2.431  loss_ce_1: 5.884  loss_mask_1: 0.1983  loss_dice_1: 2.314  loss_ce_2: 6.056  loss_mask_2: 0.2443  loss_dice_2: 2.348  loss_ce_3: 6.089  loss_mask_3: 0.2218  loss_dice_3: 2.061  loss_ce_4: 5.583  loss_mask_4: 0.2304  loss_dice_4: 2.177  loss_ce_5: 5.474  loss_mask_5: 0.1978  loss_dice_5: 2.116  loss_ce_6: 5.985  loss_mask_6: 0.1926  loss_dice_6: 2.078  loss_ce_7: 5.884  loss_mask_7: 0.2012  loss_dice_7: 1.94  loss_ce_8: 5.741  loss_mask_8: 0.1957  loss_dice_8: 1.833    time: 0.9061  last_time: 0.5307  data_time: 0.3702  last_data_time: 0.0314   lr: 0.00025  max_mem: 7801M
[05/10 13:44:56 d2.utils.events]:  eta: 0:03:31  iter: 39  total_loss: 70.67  loss_ce: 2.715  loss_mask: 0.4728  loss_dice: 3.257  loss_ce_0: 6.812  loss_mask